In [3]:
import pandas as pd
import numpy as np
from math import sqrt
from collections import Counter
import threading
pathrating = "ratings3.csv"
dtype = {"userId":np.int32, "movieId":np.int32,"rating":np.float32}
ratings = pd.read_csv(pathrating,dtype=dtype,usecols=range(3))

In [4]:
def Get_TestdData(datatrain,dtype,test):
        rating_dict = {}
        ratings = datatrain
#         ratings = pd.read_csv(pathrating,dtype=dtype,usecols=range(3))
        # Covert data to array. Get rating_dict
        for i in range(ratings.shape[0]):
                line =  ratings.iloc[i,:]
                if line.userId in rating_dict:
                        rating_dict[line.userId][line.movieId]  = line.rating
                else:
                        rating_dict[line.userId] = {line.movieId:line.rating}
        
        #make rating_dict visible
        rating_matrix = pd.DataFrame(rating_dict).T
        rating_matrix2 = pd.DataFrame(rating_dict).T
        x=rating_matrix.mean(1)

        #Linear Normalization
        rating_matrix3 = ((rating_matrix.T - rating_matrix.T.mean(axis=0))/(rating_matrix.T.max(axis=0)-rating_matrix.T.min(axis=0))).T
        #standardization Normalization
        rating_matrix4 = ((rating_matrix.T - rating_matrix.T.mean(axis=0))/rating_matrix.T.std(axis=0)).T

        # To triming data fill number 0 on places which number is None 
        rating_matrix_fill_spark_zero = rating_matrix3.fillna(0)

        #Use SVD decomposition to dimensionality reduction
        U,SIGMA,Vt = np.linalg.svd(rating_matrix_fill_spark_zero)
        reduced_matrix = (U[:,:3].dot(np.eye(3)*SIGMA[:3])).T.dot(rating_matrix_fill_spark_zero)

        #Two ways to Normalization 
        #Linear Normalization to calaulte easily
        std_matrix1 = ((reduced_matrix.T - reduced_matrix.T.mean(axis=0))/(reduced_matrix.T.max(axis=0)-reduced_matrix.T.min(axis=0))).T

        #standardization 
        std_matrix2 = ((reduced_matrix.T - reduced_matrix.T.mean(axis=0))/reduced_matrix.T.std(axis=0)).T

        #Chose Linear 
        std_matrix =std_matrix2

        #Cosine similarity between movies
        upfactor = std_matrix.T.dot(std_matrix)#分子
        downfactor = (np.linalg.norm(std_matrix,axis=0).reshape(-1,1)).dot(np.linalg.norm(std_matrix,axis=0).reshape(1,-1))
        CosSim =(upfactor/downfactor +1)/2
        
        predict = pd.DataFrame()
        movielist = rating_matrix.columns
        userlist = rating_matrix.index


        for user in range(rating_matrix.shape[0]):
            useid = userlist[user]
            isnull = np.isnan(rating_matrix.values[user,:])
            rated = ~isnull
            Recommandid = movielist[isnull]
            similar_movie_rated=(CosSim[isnull,:][:,rated])*(CosSim[isnull,:][:,rated]>0.4) # martix of 
            ratedmovie=(rating_matrix2.values[user,:][rated])
            sum_rated_cos=np.sum(similar_movie_rated,axis=1)
            #incase of factor is 0 to make factor not to be 0.
            for i in range(sum_rated_cos.shape[0]):
                if sum_rated_cos[i] == 0:
                    sum_rated_cos[i]=1
            predict_Val=similar_movie_rated.dot(ratedmovie.T)/sum_rated_cos
        #    predict_Val=(CosSim[isnull,:][:,rated]).dot(rating_matrix2.values[user,:][rated])/np.sum(CosSim[isnull,:][:,rated],axis=1)
 
            index = np.argsort(predict_Val)[::-1]
            predict_Val = predict_Val[index]
            Recommandid = Recommandid[index]
            predict = pd.concat([predict,pd.DataFrame({'userid':useid, 'RecomandID':Recommandid, 'Predice_score':predict_Val})])

        h = []

        for i in range(test.shape[0]):
            line = test.iloc[i,:]
            if line.movieId not in movielist:
                h.append(0)
            else:
                target = predict.loc[(predict['userid']==line.userId)&(predict['RecomandID']==line.movieId)].iloc[0,2]
                h.append(target)
        datatest = pd.DataFrame()
        datatest = test
        datatest['predictrate']=h
        return datatest,x

In [5]:
def k_folds(data_path, n_folds):
        data_folds_index = [[] for i in range(n_folds)]
        ratings = pd.read_csv(pathrating,dtype=dtype,usecols=range(3))
        for userid in ratings.groupby("userId").any().index:
                        user_rating_data = ratings.where(ratings["userId"]==userid).dropna()
                        index = list(user_rating_data.index.values)
                        np.random.shuffle(index)
                        start, stop = 0,0
                        for fold_i in range(n_folds):
                            start = stop
                            stop +=len(index) // n_folds
                            if fold_i < len(index) % n_folds:
                                stop +=1
                            data_folds_index[fold_i] +=(index[start:stop])
        data_folds = [ratings.loc[data_folds_index[i]] for i in range(n_folds)]
        return data_folds

In [6]:
def maeAndEmse(data,x):
    length = 0 
    _rmse_sum = 0
    _mae_sum = 0
    for i in range(data.shape[0]):
                line = data.iloc[i,:]
                if line.predictrate != 0 and (line.rating>=x[line.userId]) :
                    length += 1
                    _rmse_sum += (line.predictrate - line.rating) ** 2
                    _mae_sum += abs(line.predictrate- line.rating)
  
    return round(_mae_sum / length, 4), round(sqrt(_rmse_sum / length), 4)

In [7]:
def RmseAndmaeValue(datatrain,dtype,test):
    datatest,x =  Get_TestdData(datatrain,dtype,test)
    mae, rmse = maeAndEmse(datatest,x)
    return mae, rmse
    

In [11]:
class MyThread(threading.Thread):
    def __init__(self,func,args=()):
            super(MyThread,self).__init__()
            self.func = func
            self.args = args
    def run(self):
        self.result = self.func(*self.args)
    def get_result(self):
            try:
                return self.result # 如果子线程不使用join方法，此处可能会报没有self.result的错误
            except Exception:
                return None

In [15]:
n_folds = 5
data_folds = k_folds(pathrating,n_folds)
avg_mae = 0
avg_rmse = 0
li = []
for i in range(n_folds):
    datatrain = pd.DataFrame()
    test = pd.DataFrame()
    for j in range(n_folds):
        if i != j:
            datatrain =pd.concat([datatrain,data_folds[j]])
    test = data_folds[i].copy()   
    t = MyThread(RmseAndmaeValue,args=(datatrain,dtype,test))
    li.append(t)
    t.start()
    
for t in li:
    t.join()
    mae,rmse = t.get_result()
    avg_mae +=mae
    avg_rmse +=rmse
#      = RmseAndmaeValue(datatrain,dtype,test)


In [16]:
round(avg_mae/n_folds,4)

0.6657

In [17]:
round(avg_rmse/n_folds,4)

0.838

In [6]:
n_folds = 5
data_folds = k_folds(pathrating,n_folds)
avg_mae = 0
avg_rmse = 0
li = []
for i in range(n_folds):
    datatrain = pd.DataFrame()
    test = pd.DataFrame()
    for j in range(n_folds):
        if i != j:
            datatrain =pd.concat([datatrain,data_folds[j]])
    test = data_folds[i].copy()   
    mae, rmse =_thread.start_new_thread(RmseAndmaeValue,(datatrain,dtype,test))
#      = RmseAndmaeValue(datatrain,dtype,test)
    avg_mae +=mae
    avg_rmse +=rmse
    


In [7]:
round(avg_mae/n_folds,4)


0.6636

In [8]:
round(avg_rmse/n_folds,4)

0.8343